# Metrics

https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226

https://towardsdatascience.com/20-popular-machine-learning-metrics-part-1-classification-regression-evaluation-metrics-1ca3e282a2ce

In [1]:
import numpy as np
import json
import copy
from json import JSONEncoder
import os

In [2]:
class EvaluatedModel:
    y_pred = []
    y_true = []
    model_name = ''
    avg_time = 0.0
    gzip_size = 0
    
    accuracy = None
    precision = None
    recall = None
    f1_score = None
    

    def __init__(self, dict1=None, y_pred=[], y_true=[], model_name='', avg_time=0.0, gzip_size=0):
        if (dict1 == None):
            self.y_pred = copy.deepcopy(y_pred)
            self.y_true = copy.deepcopy(y_true)
            self.model_name = model_name
            self.avg_time = avg_time
            self.gzip_size = gzip_size
        else:
            self.__dict__.update(dict1)


## 1. Compute metrics for models

In [3]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score

def annote_models(path):
    ret_evaluated_models = []
    with open(path) as f:
        evaluated_models = json.load(f, object_hook=EvaluatedModel)

    for evaluated_model in evaluated_models:
        if len(evaluated_model.y_pred) == 0:
            continue
        evaluated_model.accuracy = accuracy_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true)
        evaluated_model.precision = precision_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='macro')
        evaluated_model.recall = recall_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='macro')
        evaluated_model.f1_score = f1_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='macro')
        # change model name for table
        evaluated_model.model_name = evaluated_model.model_name.replace("Efficent", "Efficient")
        evaluated_model.model_name = evaluated_model.model_name.replace("EfficientNetB0", "EffiNet")
        evaluated_model.model_name = evaluated_model.model_name.replace("MobileNetV2", "MobNet")
        evaluated_model.model_name = evaluated_model.model_name.replace("KMeansPlusPlus32", "WC32")
        evaluated_model.model_name = evaluated_model.model_name.replace("KMeansPlusPlus128", "WC128")
        evaluated_model.model_name = evaluated_model.model_name.replace("full_integer_quantization", "FIQ")
        evaluated_model.model_name = evaluated_model.model_name.replace("dynamic_rage_quantization", "DRQ")
        evaluated_model.model_name = evaluated_model.model_name.replace("float16_quantization", "F16Q")
        evaluated_model.model_name = evaluated_model.model_name.replace("_integer_io", "-Iio")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay90", "PD90")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay75", "PD75")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay50", "PD50")
        evaluated_model.model_name = evaluated_model.model_name.replace("flowers_model_", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("beans_model_", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_flowers_model", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_beans_model", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_tf", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_tf", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity90", "CS90")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity75", "CS75")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity50", "CS50")
        evaluated_model.model_name = evaluated_model.model_name.replace(".tflite", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_", " -> ")
        ret_evaluated_models.append(evaluated_model)
    ret_evaluated_models.sort(key=lambda x: x.model_name, reverse=False)
    return ret_evaluated_models

def print_table(models):
    row_counter = 0
    for model in models:
        if row_counter % 45 == 0:
            print('\n')
            print("|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|")
            print("|-----|-----|-----|-----|-----|-----|-----|")
        print("|"+model.model_name+"|"+str(round(model.accuracy * 100, 2))+"\%|"+str(round(model.precision * 100, 2))
              +"\%|"+str(round(model.recall * 100, 2))+"\%|"+str(round(model.f1_score * 100, 2))+"\%|"
              +str(round(model.gzip_size / 1024))+" kB|"+str(round(model.avg_time, 2))+" ms|")
        row_counter+=1


# 1. beans dataset

## 1.1 MobileNetV2

In [4]:
flowers_models = annote_models('results/evaluated_beans_models_on_MobileNetV260.json')
print_table(flowers_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|MobNet|70.31\%|72.89\%|70.41\%|70.38\%|7993 kB|230.34 ms|
|MobNet -> CS50 -> DRQ|33.59\%|11.2\%|33.33\%|16.76\%|1740 kB|387.76 ms|
|MobNet -> CS50 -> F16Q|33.59\%|11.2\%|33.33\%|16.76\%|2696 kB|228.19 ms|
|MobNet -> CS50 -> FIQ|33.59\%|11.2\%|33.33\%|16.76\%|1786 kB|167.99 ms|
|MobNet -> CS50 -> FIQ-Iio|33.59\%|11.2\%|33.33\%|16.76\%|1786 kB|249.94 ms|
|MobNet -> CS50 -> WC128 -> F16Q|33.59\%|11.2\%|33.33\%|16.76\%|2456 kB|235.31 ms|
|MobNet -> CS50 -> WC32|33.59\%|11.2\%|33.33\%|16.76\%|2691 kB|290.07 ms|
|MobNet -> CS50 -> WC32 -> DRQ|33.59\%|11.2\%|33.33\%|16.76\%|1524 kB|542.4 ms|
|MobNet -> CS50 -> WC32 -> F16Q|33.59\%|11.2\%|33.33\%|16.76\%|2125 kB|243.89 ms|
|MobNet -> CS75|33.59\%|11.2\%|33.33\%|16.76\%|3133 kB|232.51 ms|
|MobNet -> CS75 -> WC128|33.59\%|11.2\%|33.33\%|16.76\%|2400 kB|229.61 ms|
|MobNet -> CS75 -> WC128 -> DRQ|33.59\%|11.2\%|33.33\%|16.76\%|1215 kB|37

/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 1.2 EfficientNetB0

In [5]:
beans_models = annote_models('results/evaluated_beans_models_on_EfficentNetB0.json')
print_table(beans_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|EffiNet|73.44\%|75.0\%|73.39\%|73.58\%|14488 kB|2476.57 ms|
|EffiNet -> CS50|33.59\%|11.2\%|33.33\%|16.76\%|9163 kB|2586.88 ms|
|EffiNet -> CS50 -> DRQ|33.59\%|11.2\%|33.33\%|16.76\%|3189 kB|2491.33 ms|
|EffiNet -> CS50 -> F16Q|33.59\%|11.2\%|33.33\%|16.76\%|5015 kB|2587.94 ms|
|EffiNet -> CS50 -> FIQ|33.59\%|11.2\%|33.33\%|16.76\%|3264 kB|1954.81 ms|
|EffiNet -> CS50 -> WC128|32.81\%|10.94\%|33.33\%|16.47\%|6327 kB|2527.01 ms|
|EffiNet -> CS50 -> WC128 -> DRQ|32.81\%|10.94\%|33.33\%|16.47\%|3104 kB|2566.19 ms|
|EffiNet -> CS50 -> WC128 -> F16Q|32.81\%|10.94\%|33.33\%|16.47\%|4419 kB|2622.47 ms|
|EffiNet -> CS50 -> WC128 -> FIQ|32.81\%|10.94\%|33.33\%|16.47\%|3165 kB|2097.07 ms|
|EffiNet -> CS50 -> WC32|32.81\%|14.45\%|32.56\%|18.49\%|4719 kB|2500.41 ms|
|EffiNet -> CS50 -> WC32 -> DRQ|32.81\%|11.38\%|32.56\%|16.87\%|2678 kB|2733.59 ms|
|EffiNet -> CS50 -> WC32 -> F16Q|32.81\

# 2. TF Flowers

## 2.1 MobileNetV2

In [6]:
tf_flowers_models = annote_models('results/evaluated_tf_flowers_models_on_MobileNetV2.json')
print_table(tf_flowers_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|MobNet|66.73\%|70.28\%|68.53\%|67.22\%|8463 kB|62.81 ms|
|MobNet -> CS50|20.36\%|4.07\%|20.0\%|6.77\%|5284 kB|62.61 ms|
|MobNet -> CS50 -> DRQ|20.36\%|4.07\%|20.0\%|6.77\%|1841 kB|75.58 ms|
|MobNet -> CS50 -> F16Q|20.36\%|4.07\%|20.0\%|6.77\%|2895 kB|62.41 ms|
|MobNet -> CS50 -> FIQ|20.36\%|4.07\%|20.0\%|6.77\%|1884 kB|46.45 ms|
|MobNet -> CS50 -> FIQ-Iio|20.36\%|4.07\%|20.0\%|6.77\%|1884 kB|47.26 ms|
|MobNet -> CS50 -> WC128|20.36\%|4.07\%|20.0\%|6.77\%|3808 kB|62.18 ms|
|MobNet -> CS50 -> WC128 -> DRQ|20.36\%|4.07\%|20.0\%|6.77\%|1820 kB|75.85 ms|
|MobNet -> CS50 -> WC128 -> F16Q|20.36\%|4.07\%|20.0\%|6.77\%|2613 kB|62.59 ms|
|MobNet -> CS50 -> WC128 -> FIQ|20.36\%|4.07\%|20.0\%|6.77\%|1864 kB|46.42 ms|
|MobNet -> CS50 -> WC128 -> FIQ-Iio|20.36\%|4.07\%|20.0\%|6.77\%|1864 kB|47.12 ms|
|MobNet -> CS50 -> WC32|20.36\%|4.07\%|20.0\%|6.77\%|2787 kB|62.75 ms|
|MobNet -> CS50 -> 

## 2.2 EfficientNetB0

In [7]:
tf_flowers_models = annote_models('results/evaluated_tf_flowers_models_on_EfficentNetB0.json')
print_table(tf_flowers_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|EffiNet|70.91\%|71.36\%|71.11\%|70.85\%|14923 kB|559.65 ms|
|EffiNet -> CS50|54.55\%|60.37\%|54.01\%|53.23\%|9479 kB|585.05 ms|
|EffiNet -> CS50 -> DRQ|55.09\%|61.59\%|54.39\%|53.64\%|3287 kB|570.78 ms|
|EffiNet -> CS50 -> F16Q|54.55\%|60.37\%|54.01\%|53.23\%|5209 kB|589.29 ms|
|EffiNet -> CS50 -> FIQ|56.36\%|61.9\%|55.78\%|55.16\%|3356 kB|567.24 ms|
|EffiNet -> CS50 -> WC128|71.64\%|71.48\%|72.35\%|71.54\%|6798 kB|583.22 ms|
|EffiNet -> CS50 -> WC128 -> DRQ|72.55\%|72.39\%|73.2\%|72.43\%|3211 kB|618.9 ms|
|EffiNet -> CS50 -> WC128 -> F16Q|71.64\%|71.48\%|72.35\%|71.54\%|4709 kB|597.82 ms|
|EffiNet -> CS50 -> WC128 -> FIQ|71.82\%|71.53\%|72.4\%|71.61\%|3265 kB|472.23 ms|
|EffiNet -> CS50 -> WC32|70.18\%|73.09\%|69.95\%|68.72\%|5160 kB|575.71 ms|
|EffiNet -> CS50 -> WC32 -> DRQ|69.09\%|72.15\%|68.8\%|67.12\%|2786 kB|595.56 ms|
|EffiNet -> CS50 -> WC32 -> F16Q|70.18\%|73.09\%|6

# 3. Oxford Flowers

## 3.1 MobileNetV2

In [11]:
flowers_models = annote_models('results/evaluated_flowers_models_on_MobileNetV290.json')
print_table(flowers_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|MobNet -> CS50|0.33\%|0.0\%|0.98\%|0.01\%|5277 kB|63.14 ms|
|MobNet -> CS50 -> DRQ|0.33\%|0.0\%|0.98\%|0.01\%|1835 kB|76.3 ms|
|MobNet -> CS50 -> F16Q|0.33\%|0.0\%|0.98\%|0.01\%|2888 kB|62.55 ms|
|MobNet -> CS50 -> FIQ|0.33\%|0.0\%|0.98\%|0.01\%|1882 kB|51.77 ms|
|MobNet -> CS50 -> FIQ-Iio|0.33\%|0.0\%|0.98\%|0.01\%|1882 kB|47.8 ms|
|MobNet -> CS50 -> WC128|0.75\%|0.01\%|0.98\%|0.01\%|3805 kB|62.74 ms|
|MobNet -> CS50 -> WC128 -> DRQ|0.75\%|0.01\%|0.98\%|0.01\%|1835 kB|78.77 ms|
|MobNet -> CS50 -> WC128 -> F16Q|0.75\%|0.01\%|0.98\%|0.01\%|2597 kB|62.64 ms|
|MobNet -> CS50 -> WC32|0.75\%|0.01\%|0.98\%|0.01\%|2800 kB|63.02 ms|
|MobNet -> CS50 -> WC32 -> DRQ|0.75\%|0.01\%|0.98\%|0.01\%|1567 kB|86.21 ms|
|MobNet -> CS50 -> WC32 -> F16Q|0.75\%|0.01\%|0.98\%|0.01\%|2214 kB|62.65 ms|
|MobNet -> CS75|1.63\%|0.02\%|0.98\%|0.03\%|3332 kB|62.73 ms|
|MobNet -> CS75 -> DRQ|1.63\%|0.02\%|0

## 3.2 EfficientNetB0

In [12]:
flowers_models = annote_models('results/evaluated_flowers_models_on_EfficentNetB0.json')
print_table(flowers_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|EffiNet|14.77\%|14.68\%|16.36\%|12.53\%|14937 kB|562.42 ms|
|EffiNet -> CS50|0.39\%|0.0\%|0.98\%|0.01\%|9479 kB|547.46 ms|
|EffiNet -> CS50 -> DRQ|0.39\%|0.0\%|0.98\%|0.01\%|3286 kB|627.6 ms|
|EffiNet -> CS50 -> F16Q|0.39\%|0.0\%|0.98\%|0.01\%|5200 kB|561.71 ms|
|EffiNet -> CS50 -> FIQ|0.39\%|0.0\%|0.98\%|0.01\%|3358 kB|515.01 ms|
|EffiNet -> CS50 -> WC128|6.7\%|3.18\%|5.83\%|2.74\%|6803 kB|587.03 ms|
|EffiNet -> CS50 -> WC128 -> DRQ|6.96\%|3.44\%|6.36\%|3.07\%|3215 kB|591.12 ms|
|EffiNet -> CS50 -> WC128 -> F16Q|6.7\%|3.2\%|5.83\%|2.74\%|4670 kB|622.4 ms|
|EffiNet -> CS50 -> WC128 -> FIQ|6.64\%|3.22\%|5.75\%|2.77\%|3270 kB|523.37 ms|
|EffiNet -> CS50 -> WC32|2.47\%|0.16\%|2.66\%|0.29\%|5176 kB|618.94 ms|
|EffiNet -> CS50 -> WC32 -> DRQ|2.8\%|0.56\%|2.66\%|0.53\%|2795 kB|623.9 ms|
|EffiNet -> CS50 -> WC32 -> F16Q|2.47\%|0.16\%|2.66\%|0.29\%|3987 kB|603.15 ms|
|EffiNet -> CS50